In [1]:
import json

import os
import openai

from tqdm import tqdm
import time

import numpy as np
np.random.seed(0)

openai.api_key = os.getenv("OPENAI_API_KEY")

dev = json.load(open("../data/individual_dev_end2end_final.json"))

get_lemma = lambda x:" ".join([token.lemma_ for token in nlp(x)])


# nlp = spacy.load("en_core_web_sm")

relation_trigger = ['before', 'after', 'during', 'while']

warmup_qs = ['What will happen in the future?', 'What event has already finished?', 
             'What event has begun but has not finished?', 'What is happening now?',
             'What event has already happened?', 'What event has started?']

In [3]:
zero_shot_results = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + "}\nA:"
    
    zero_shot_results[key] = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              max_tokens=256,
              temperature=0
    )["choices"][0]["text"].strip()
    
    time.sleep(2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [1:18:13<00:00,  3.16s/it]


In [4]:
from eval_func_gpt3 import evaluate
evaluate(zero_shot_results, dev)

1483it [00:00, 25740.34it/s]


Total 1483 questions
Eval on the current eval positive class Micro F1 (Agg) is: 0.4179
Eval on the current eval positive class Macro F1 (Relaxed) is: 0.4590
Eval on the current eval positive class Macro F1 (Agg) is: 0.3806
Eval on the current eval exact match (Agg) ratio is: 0.0593
Eval on the current eval exact match ratio (Relaxed) is: 0.0816
Eval on 485 Clusters
Eval on the current eval clustered EM (Agg) is: 0.0041
Eval on the current eval clustered EM (Relaxed) is: 0.0041
Eval on the current eval clusrered F1 (max>=0.8) is: 0.0124


# one shot

In [5]:
train = json.load(open("../data/train_end2end_final.json"))

In [12]:
import numpy as np
np.random.seed(1)

random_key = np.random.choice(list(train.keys()), 1)[0]
print(random_key)
item = train[random_key]
print(item)

context = " ".join(item['context'])
question = item['question']
all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
# examplar_prompt_0 = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
examplar_prompt_0 = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + f"\nA: {', '.join(ground)}"
print(examplar_prompt_0)


docid_AFP_ENG_19970401.0534_sentid_6_39DD6S19JQ0Q8KWDBW0CSU5CC44ZE7_3
{'answers': {'labels': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'types': [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'context': ['A', 'public', 'security', 'bureau', 'official', 'said', 'the', 'police', 'were', 'unable', 'to', 'comment', 'on', 'the', 'statement', '.', 'But', 'Shanghai', 'police', 'did', 'say', 'that', 'on', 'March', '24', 'they', 'had', 'taken', 'similar', 'action', 'against', 'Bishop', 'Joseph', 'Fan', 'Zhongliang', ',', 'coadjutor', 'of', 'the', 'underground', 'Roman', 'Catholic', 'Church', '.'], 'question': 'What happened prior to the public security bureau official speaking?'}
Given the context A public security bureau official said the police were unable to comment on the statement . But Shanghai police did say tha

In [13]:
one_shot_results_0 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + "}\nA:"
    
    while True:
        try:
            one_shot_results_0[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_prompt_0 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [50:09<00:00,  2.03s/it]


In [22]:
with open('data/template_2_oneshot_exemplar_0.txt', 'w') as writer:
    writer.writelines(examplar_prompt_0)
with open('results/template_2_oneshot_0_pred', 'w') as writer:
    json.dump(one_shot_results_0, writer)

In [16]:
evaluate(one_shot_results_0, dev)

1483it [00:00, 38033.98it/s]


Total 1483 questions
Eval on the current eval positive class Micro F1 (Agg) is: 0.4620
Eval on the current eval positive class Macro F1 (Relaxed) is: 0.5048
Eval on the current eval positive class Macro F1 (Agg) is: 0.4261
Eval on the current eval exact match (Agg) ratio is: 0.0668
Eval on the current eval exact match ratio (Relaxed) is: 0.0951
Eval on 485 Clusters
Eval on the current eval clustered EM (Agg) is: 0.0041
Eval on the current eval clustered EM (Relaxed) is: 0.0041
Eval on the current eval clusrered F1 (max>=0.8) is: 0.0247


In [11]:
np.random.seed(2)

random_key = np.random.choice(list(train.keys()), 1)[0]
print(random_key)
item = train[random_key]
print(item)

context = " ".join(item['context'])
question = item['question']
all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
# examplar_prompt_0 = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
examplar_prompt_1 = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + f"\nA: {', '.join(ground)}"
print(examplar_prompt_1)


docid_wsj_0791_sentid_7_3N8OEVH1FV4K14LQJ53CXLQVB2SOOE_1
{'answers': {'labels': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'types': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]}, 'context': ['Norwood', 'is', 'controlled', 'by', 'Daniel', 'L.', 'Barnett', 'and', 'Paul', 'A.', 'Reese', ',', 'both', 'officers', 'of', 'Boston-based', 'Oasis', 'Capital', 'Management', 'Inc.', ',', 'a', 'small', 'Boston', 'money', 'management', 'firm', '.', 'Also', 'involved', 'in', 'the', 'group', 'is', 'Robert', 'F.', 'Angelo', ',', 'formerly', 'Phoenix', "'s", 'senior', 'vice', 'president', ',', 'field', 'operations', ',', 'who', 'left', 'Phoenix', 'at', 'the', 'beginning', 'of', 'October', '.'], 'question': 'What is happening now?'}
Given the context N

In [14]:
one_shot_results_1 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + "}\nA:"
    
    while True:
        try:
            one_shot_results_1[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_prompt_1 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [47:57<00:00,  1.94s/it]


In [18]:
evaluate(one_shot_results_1, dev)

1483it [00:00, 52636.88it/s]


Total 1483 questions
Eval on the current eval positive class Micro F1 (Agg) is: 0.4654
Eval on the current eval positive class Macro F1 (Relaxed) is: 0.5127
Eval on the current eval positive class Macro F1 (Agg) is: 0.4348
Eval on the current eval exact match (Agg) ratio is: 0.0931
Eval on the current eval exact match ratio (Relaxed) is: 0.1261
Eval on 485 Clusters
Eval on the current eval clustered EM (Agg) is: 0.0082
Eval on the current eval clustered EM (Relaxed) is: 0.0082
Eval on the current eval clusrered F1 (max>=0.8) is: 0.0268


In [23]:
with open('data/template_2_oneshot_exemplar_1.txt', 'w') as writer:
    writer.writelines(examplar_prompt_1)
with open('results/template_2_oneshot_1_pred', 'w') as writer:
    json.dump(one_shot_results_1, writer)

In [19]:
np.random.seed(3)

random_key = np.random.choice(list(train.keys()), 1)[0]
print(random_key)
item = train[random_key]
print(item)

context = " ".join(item['context'])
question = item['question']
all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
# examplar_prompt_0 = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
examplar_prompt_2 = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + f"\nA: {', '.join(ground)}"
print(examplar_prompt_2)


docid_AFP_ENG_20051212.0486_sentid_6_3TYCR1GOTD84EQ65SING5PVWKW6ZLK_2
{'answers': {'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'types': [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]}, 'context': ['About', '59', 'percent', 'of', 'eligible', 'Iraqis', 'cast', 'ballots', 'last', 'January', 'for', 'an', 'interim', 'parliament', ',', 'and', '64', 'percent', 'voted', 'in', 'the', 'October', '15', 'referendum', 'on', 'a', 'new', 'constitution', '.', 'The', 'US', 'envoy', 'said', 'that', 'unlike', 'the', 'runup', 'to', 'January', "'s", 'ballot', ',', '@', 'the', 'level', 'of', 'violence', 'has', 'dropped', 'my', 'almost', 'any', 'measure', 'over', 'the', 'past', 'few', 'days

In [20]:
one_shot_results_2 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + "}\nA:"
    
    while True:
        try:
            one_shot_results_2[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_prompt_2 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [50:16<00:00,  2.03s/it]


In [21]:
evaluate(one_shot_results_2, dev)

1483it [00:00, 40147.63it/s]


Total 1483 questions
Eval on the current eval positive class Micro F1 (Agg) is: 0.4353
Eval on the current eval positive class Macro F1 (Relaxed) is: 0.4885
Eval on the current eval positive class Macro F1 (Agg) is: 0.4094
Eval on the current eval exact match (Agg) ratio is: 0.0755
Eval on the current eval exact match ratio (Relaxed) is: 0.1052
Eval on 485 Clusters
Eval on the current eval clustered EM (Agg) is: 0.0041
Eval on the current eval clustered EM (Relaxed) is: 0.0082
Eval on the current eval clusrered F1 (max>=0.8) is: 0.0206


In [24]:
with open('data/template_2_oneshot_exemplar_2.txt', 'w') as writer:
    writer.writelines(examplar_prompt_2)
with open('results/template_2_oneshot_2_pred', 'w') as writer:
    json.dump(one_shot_results_2, writer)

# threeshot

In [ ]:
train = json.load(open("../data/train_end2end_final.json"))
import numpy as np
np.random.seed(1)

random_keys = np.random.choice(list(train.keys()), 3)
print(random_keys)
items = [train[k] for k in random_keys]
print(items)


examplar_list = []
for item in items:
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    if len(ground) == 0:
        ground = ['none']
    tmp_prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + f"\nA: {', '.join(ground)}"
    examplar_list.append(tmp_prompt)
examplar_threeshot_0 = "\n\n".join(examplar_list)


In [ ]:
three_shot_results_0 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + "}\nA:"
    
    while True:
        try:
            three_shot_results_0[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_threeshot_0 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

In [ ]:
train = json.load(open("../data/train_end2end_final.json"))
import numpy as np
np.random.seed(2)

random_keys = np.random.choice(list(train.keys()), 3)
print(random_keys)
items = [train[k] for k in random_keys]
print(items)


examplar_list = []
for item in items:
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    if len(ground) == 0:
        ground = ['none']
    tmp_prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + f"\nA: {', '.join(ground)}"
    examplar_list.append(tmp_prompt)
examplar_threeshot_1 = "\n\n".join(examplar_list)


In [ ]:
three_shot_results_1 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + "}\nA:"
    
    while True:
        try:
            three_shot_results_1[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_threeshot_1 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

In [ ]:
train = json.load(open("../data/train_end2end_final.json"))
import numpy as np
np.random.seed(3)

random_keys = np.random.choice(list(train.keys()), 3)
print(random_keys)
items = [train[k] for k in random_keys]
print(items)


examplar_list = []
for item in items:
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    if len(ground) == 0:
        ground = ['none']
    tmp_prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + f"\nA: {', '.join(ground)}"
    examplar_list.append(tmp_prompt)
examplar_threeshot_2 = "\n\n".join(examplar_list)


In [ ]:
three_shot_results_2 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + "}\nA:"
    
    while True:
        try:
            three_shot_results_2[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_threeshot_2 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)